In [ ]:
import pandas as pd
import tensorflow as tf
import numpy as np
import TD3_tf2
import ForexEnv
import preprocess
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (17,10)

### Pre-trained Model Performance on user's data  

This notebook is for assessing the sample pre-trained model's performance when being implemented on user's broker EUR/USD daily price data.<br><br>  

The model was trained using 2012-3-5 to 2019-3-20 EUR/USD daily price data with closing prices set at 00:00:00 GMT, except Friday.  

Backtesting Period: 2019-3-21 onwards<br><br>  

Before backtesting its performance, user should provide:  
<br>
#### EUR/USD historical price dataset    
```backtest_historical```, an unprocessed price dataset that must contain prices for defining trade entries and exits for reward calculation (pandas DataFrame expected).  

e.g. ```backtest_historical = pd.read_csv('EURUSD_daily.csv')```  

- ```backtest_historical.columns``` are expected to be ```['Bid Open', 'Bid High', 'Bid Low', 'Bid Close', 'Ask Open', 'Ask High', 'Ask Low', 'Ask Close']```.<br><br>  

- ```backtest_historical.index``` is expected to be dates, where trade entries and exits are set to be the close prices of each day (BidClose and AskClose).<br><br>  

    - Note: For preprocessing purposes, the first 55 trading days of the dataframe will not be visible to the trading RL agent. 
    - For example, to have the backtesting period starts on 2019-3-21, the provided dataset should contain price data starting on 2019-1-17 as shown:<br><br>  

    | Index | Bid Open | Bid High | Bid Low | Bid Close | Ask Open | Ask High | Ask Low | Ask Close |
    | :---: | :------: | :------: | :-----: | :-------: | :------: | :------: | :-----: | :-------: |
    | 2019-01-17 | ... | ... | ... | ... | ... | ... | ... | ... |
    | 2019-01-18 | ... | ... | ... | ... | ... | ... | ... | ... |
    |     ...    | ... | ... | ... | ... | ... | ... | ... | ... |
    | 2021-10-21 | ... | ... | ... | ... | ... | ... | ... | ... |

<br>

In [ ]:
# (user-input)
backtest_historical = 

In [ ]:
test_historical = backtest_historical.copy()
test_agent_obs = test_historical.copy()
test_agent_obs.columns = ['Open', 'High', 'Low', 'Close', 'AskOpen', 'AskHigh', 'AskLow', 'AskClose']
test_agent_obs = preprocess.ma(df=test_agent_obs, ma_ranges=[10, 21, 50])
test_agent_obs = preprocess.macd(df=test_agent_obs, short_long_signals=[(12, 26, 9)])
test_agent_obs = preprocess.full_stochastic(df=test_agent_obs, stochastic_ranges=[(14, 3, 3)])
test_agent_obs = preprocess.rsi(df=test_agent_obs, rsi_ranges=[14])
test_agent_obs = preprocess.bollinger_bands(df=test_agent_obs,
                                            bollinger_period_sd_ranges=[(20, 2)])
test_agent_obs = test_agent_obs.iloc[:, 8:]
test_agent_obs = preprocess.add_lag(df=test_agent_obs, lag=5)
test_agent_obs = test_agent_obs.dropna()

mins = np.load("samples/sample_mins.npy")
maxs = np.load("samples/sample_maxs.npy")

test_agent_obs = test_agent_obs.clip(mins, maxs, axis=1)

test_agent_obs = (test_agent_obs - mins) / (maxs - mins)

test_historical = np.array(test_historical.loc[test_agent_obs.index])
test_agent_obs = np.array(test_agent_obs)

env = ForexEnv.ForexEnv(agent_obs_arrays=test_agent_obs, bidask_arrays=test_historical,
                        initial_balance=1, bid_col=3, 
                        ask_col=7, p=0.1, n_actions=1)

final_actor = TD3_tf2.ActorNetwork(fc_dim=1024, num_layers=4, n_actions=1)
_ = final_actor(tf.ones((1, 74)))
final_actor.load_weights("samples/sample_actor.h5")

history = {}

observation, reward_state = env.reset(evaluate=True)
done = False
score_list = []
balance_list = []
action_list = []
starting_position = 0
while not done:
    action = final_actor(observation)
    action_list.append(action[0,0].numpy())
    observation_, reward, done, reward_state_, info = env.step(action, reward_state)
    score_list.append(np.reshape(reward,(1,))[0])
    balance_list.append(np.reshape(env.balance,(1,))[0])
    observation = observation_
    reward_state = reward_state_

history["Pre-trained Model Performance Score"] = score_list
history["Pre-trained Model Performance Balance"] = balance_list
print(f"Pre-trained Model Performance \n\
Starting Position {starting_position} \n\
Ending Position {env.current_pos} \n\
Ending Balance {env.balance[0,0].numpy():.3f} \n\
Avg_reward {(env.balance[0,0].numpy()) ** (1 / (env.current_pos - starting_position)) - 1}")

plt.plot(np.array(balance_list))
plt.xlabel("Timestamps")
plt.ylabel("Balance (Initial Balance = 1, i.e. 100%)")
plt.title(f"Pre-trained Model Performance on user's data")
plt.show()

<br>  

*The Pre-trained model was trained on price data provided by a broker that may or may not be different to which of the user's, the performance and the estimated action may and is likely to vary.*<br><br>  